<h1>Datascience Capstone Project</h1>

In this capstone project, hopefully I will learn new skill related location data and how to use it develop cool applciations.

Import the Following Libraries:
<ul>
    <li> <b>numpy (as np)</b> </li>
    <li> <b>pandas</b> </li>
</ul>

In [3]:
import numpy as np 
import pandas as pd

Hello Capstone Project Course!